In [3]:
import os
import psycopg2
import pandas as pd
import numpy as np
import numba as nb
import pickle
import logging
from tqdm.notebook import tqdm
from datetime import datetime
import pytz
from scipy.interpolate import interp1d

from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits
from astropy.time import Time
from astropy.table import Table
from astropy.timeseries import aggregate_downsample, TimeSeries
import astropy.units as u

data_dir = '../../py/desiforecast/data'

In [2]:
with open(f'{data_dir}/environmentmonitor_tower.pkl', 'rb') as pf:
    tower_data = pd.read_pickle(pf)
with open(f'{data_dir}/environmentmonitor_dome.pkl', 'rb') as pf:
    dome_data = pd.read_pickle(pf)
with open(f'{data_dir}/environmentmonitor_telescope.pkl', 'rb') as pf:
    telescope_data = pd.read_pickle(pf)
with open(f'{data_dir}/etc_seeing.pkl', 'rb') as pf:
    seeing_data = pd.read_pickle(pf)
with open(f'{data_dir}/etc_telemetry.pkl', 'rb') as pf:
    telemetry_data = pd.read_pickle(pf)
with open(f'{data_dir}/tcs_info.pkl', 'rb') as pf:
    tcs_data = pd.read_pickle(pf)
telemetry_data.sort_index(inplace=True)
seeing_data.sort_index(inplace=True)


KeyboardInterrupt



In [ ]:
tower_data = TimeSeries.from_pandas(tower_data)
dome_data = TimeSeries.from_pandas(dome_data)
telescope_data = TimeSeries.from_pandas(telescope_data)
seeing_data = TimeSeries.from_pandas(seeing_data)
telemetry_data = TimeSeries.from_pandas(telemetry_data)
tcs_data = TimeSeries.from_pandas(tcs_data)

In [39]:
tower_data.write(f'{data_dir}/environmentmonitor_tower.fits', overwrite=True)
dome_data.write(f'{data_dir}/environmentmonitor_dome.fits', overwrite=True)
telescope_data.write(f'{data_dir}/environmentmonitor_telescope.fits', overwrite=True)
seeing_data.write(f'{data_dir}/etc_seeing.fits', overwrite=True)
telemetrydata.write(f'{data_dir}/etc_telemetry.fits', overwrite=True)
tcs_data.write(f'{data_dir}/tcs_info.fits', overwrite=True)


ValueError: cannot replace a table index column

In [ ]:
print(tower_data)

In [97]:
# def preprocess_data(data):
#     nans = np.isnan(y)
#     x = lambda z: z.nonzero()[0]
#     data[nans] = np.interp(x(nans), x(~nans), y[~nans])
#     data
#     if 'seeing' in data.colnames:
#         pass
#     else:
#         data = aggregate_downsample(data, time_bin_size=6*u.s)
#     return data
def preprocess_data(rows, columns):
    """Preprocesses DESI telemetry data contained within Pandas dataframe.
    
    Parameters
    ----------
    rows : numpy.ndarray
        Unprocessed data as queried from the DESI telemetry database
    
    Returns
    -------
    data : astropy.io.fits.BinTableHDU
        Resampled and interpolated DESI telemetry data
    """
    data = pd.DataFrame(rows, columns=columns)
    data.set_index('time_recorded', inplace=True)
    data.index.name = 'time'
    if 'seeing' in data.columns.values:
        data.fillna(0.0, inplace=True)
    else:
        data = data.resample('6S').mean()
        data.interpolate(limit_direction='both', inplace=True)
    for i in data.columns.values:
        if data[i].dtypes == 'object':
            data[i] = data[i].astype(float)
    data.reset_index(inplace=True)
    data['time'] = Time(data['time'])
    print(data.dtypes)
    return fits.table_to_hdu(Table.from_pandas(data))

In [99]:
labels = {}
labels['etc_seeing'] = ['time_recorded', 'seeing']

host = 'replicator-db.desi.lbl.gov'
database = 'desi_dev'
user = 'desi_reader'
password = 'reader'
port = '5432'

for table, columns in tqdm(labels.items(), desc='Querying database tables...'):
    conn = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)
    print(f'{table}:')
    with conn:
        with conn.cursor() as cur:
            print('\tQuerying database table...')
            cur.execute(f'SELECT {", ".join(columns)} FROM {table};')
            rows = np.asarray(cur.fetchall())
            data = preprocess_data(rows, columns)
            data.writeto(f'{data_dir}/{table}.fits', overwrite=True)

Querying database tables...:   0%|          | 0/1 [00:00<?, ?it/s]

etc_seeing:
	Querying database table...
time       object
seeing    float64
dtype: object


ValueError: Illegal format `object`.

In [4]:
data = Table.read(f'{data_dir}/etc_seeing.fits')

In [5]:
for i in range(len(data)):
    if data[i][0]!='2':
        print(data[i][0])

In [188]:
data.write(f'{data_dir}/etc_seeing.fits', overwrite=True)